In [31]:
import pandas as pd
import numpy as np
from datetime import datetime
import requests
import re

# Collect and Clean Twitter Data

#### Using http://www.trumptwitterarchive.com/archive , I downloaded all of Donald Trump's tweets from 01/01/2016 - 10/11/2017 9:19 am MST

In [1]:
# filepath to twitter archive data.
twitter_json = r'C:\Users\aregel\Documents\springboard\Capstone_2\data_wrangle\data\twitter_01_01_16_to_10-11-17.json' 

### Using Pandas I will read the twitter json file, convert it to a dataframe, set the index to 'created at' as datetime objects, then write it to a csv

In [20]:
# read the json data into a pandas dataframe
tweet_data = pd.read_json(twitter_json)
# set column 'created_at' to the index
tweet_data.set_index('created_at', drop=True, inplace= True)
# convert timestamp index to a datetime index
pd.to_datetime(tweet_data.index)
# write to csv file
csv_file_path = r'C:\Users\aregel\Documents\springboard\Capstone_2\data_wrangle\data\twitter_01_01_16_to_10-11-17.csv' 
tweet_data.to_csv(csv_file_path)

# Scrape Data from the Federal Register

In [47]:
# Define the 2016, 2017 url that contains all of the Executive Office of the President's published documents
executive_office_url_2016 = r'https://www.federalregister.gov/index/2016/executive-office-of-the-president'
executive_office_url_2017 = r'https://www.federalregister.gov/index/2017/executive-office-of-the-president'

# scrape both websites' html to one list of url's containing the pdf documents
pdf_urls = []
for url in [executive_office_url_2016, executive_office_url_2017]:
    response = requests.get(url)
    pattern = re.compile(r'https:.*\.pdf')
    pdfs = re.findall(pattern, text)
    pdf_urls.append(pdfs)
    

In [48]:
pdf_urls

[['https://www.gpo.gov/fdsys/pkg/FR-2016-02-29/pdf/2016-04291.pdf',
  'https://www.gpo.gov/fdsys/pkg/FR-2016-04-21/pdf/2016-09230.pdf',
  'https://www.gpo.gov/fdsys/pkg/FR-2016-04-13/pdf/2016-08713.pdf',
  'https://www.gpo.gov/fdsys/pkg/FR-2016-08-26/pdf/2016-20713.pdf',
  'https://www.gpo.gov/fdsys/pkg/FR-2016-05-23/pdf/2016-12307.pdf',
  'https://www.gpo.gov/fdsys/pkg/FR-2016-02-12/pdf/2016-03170.pdf',
  'https://www.gpo.gov/fdsys/pkg/FR-2016-06-14/pdf/2016-14224.pdf',
  'https://www.gpo.gov/fdsys/pkg/FR-2016-04-15/pdf/2016-08970.pdf',
  'https://www.gpo.gov/fdsys/pkg/FR-2016-06-03/pdf/2016-13290.pdf',
  'https://www.gpo.gov/fdsys/pkg/FR-2016-02-02/pdf/2016-02084.pdf',
  'https://www.gpo.gov/fdsys/pkg/FR-2016-05-18/pdf/2016-11936.pdf',
  'https://www.gpo.gov/fdsys/pkg/FR-2016-10-12/pdf/2016-24847.pdf',
  'https://www.gpo.gov/fdsys/pkg/FR-2016-11-14/pdf/2016-27498.pdf',
  'https://www.gpo.gov/fdsys/pkg/FR-2016-02-18/pdf/2016-03540.pdf',
  'https://www.gpo.gov/fdsys/pkg/FR-2016-05-11/p

In [58]:
url2 = r'https://www.gpo.gov/fdsys/pkg/FR-2016-11-08/pdf/2016-27156.pdf/'

In [59]:
r = requests.get(url2)
r

<Response [404]>

In [ ]:
response = requests.get(url2, stream=True)
handle = open(target_path, "wb")
for chunk in response.iter_content(chunk_size=512):
    if chunk:  # filter out keep-alive new chunks
        handle.write(chunk)

In [55]:
with open("python.pdf","wb") as pdf:
    for chunk in r.iter_content(chunk_size=1024):
 
         # writing one chunk at a time to pdf file
         if chunk:
             pdf.write(chunk)

TypeError: invalid file: <Response [200]>